In [230]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import time
from tqdm import tqdm
import matplotlib.pyplot as plt #not needed to run program
import datetime
import logging
from pypfopt import risk_models, expected_returns, EfficientFrontier, objective_functions
import itertools
import warnings
import plotly.express as px

In [65]:
logging.basicConfig(format = '%(asctime)s:%(levelname)s :%(message)s',
                        datefmt = '%Y-%m-%d %H:%M:%S',
                        filename = 'logs/run_main.log',
                        level=logging.INFO)

**1. Get current DAX constituents from wikipedia**

In [66]:
def replace_by_freq(dataframe, column_name, threshold, new_value):

    """
    Replace values in a column of a DataFrame based on their frequency count.

    Args:
        dataframe (pandas.DataFrame): The DataFrame to modify.
        column_name (str): The name of the column to replace values in.
        threshold (int): The frequency threshold. Values appearing less than this threshold will be replaced.
        new_value: The new value to replace the infrequent values with.

    Returns:
        pandas.DataFrame: The modified DataFrame with values replaced.

    Raises:
        KeyError: If the specified column is not found in the DataFrame.
        
    """

    try:
        unique = dataframe.groupby(column_name)[column_name].transform('size').lt(threshold)
        dataframe.loc[unique, column_name] = new_value
    except KeyError as ke:
        print(f'Column ', ke, 'is not found in a dataframe')
    
    return dataframe

In [67]:
def dax40_wikipedia_constituents():
    
    """
    Retrieve the constituents of the DAX40 index from Wikipedia (https://en.wikipedia.org/wiki/DAX).

     Args:
       None

    Returns:
        pandas.DataFrame: DataFrame containing the DAX40 constituents.

    Raises:
        Exception: If the website URL is unreachable or does not exist.
        Exception: If the 'constituents' table cannot be found on the website.
        Exception: If there is an error while extracting columns from the table.
        KeyError: If there is an issue with the column index of the 'constituents' table.

    """

    url = 'https://en.wikipedia.org/wiki/DAX'

    #Check if the connection is succesful
    try:
        response = requests.get(url)
        if response.ok:
            soup = BeautifulSoup(response.text, 'html.parser')
        else:
            raise Exception(f'Cannot reach website {url}')
    except:
        raise Exception(f'Website {url} does not exist')

    #check if table 'constituents' exists
    wiki_table_id = 'constituents'
    try:
        indiatable = soup.find('table',{'id': wiki_table_id,'class':'wikitable'})
        df = pd.read_html(str(indiatable))
    except:
        raise Exception(f'Table {wiki_table_id} cannot be found in {url}')

    #Extract columns from table
    extract_cols = ['Ticker', 'Company', 'Prime Standard Sector']
    try:
        dax_constituents = pd.DataFrame(df[0])[extract_cols]
        dax_constituents = dax_constituents.rename(columns={'Prime Standard Sector': 'Sector', 'Company': 'Name'})
    except KeyError as e:
        raise Exception((e.args[0]).replace('index', 'column index of table constituents'))

    dax_constituents = replace_by_freq(dax_constituents, 'Sector', 2, 'Other')

    logging.info(f'DAX40 constituents loaded successfully')

    return dax_constituents

In [68]:
dax_constituents_info = dax40_wikipedia_constituents()

In [70]:
# dax_constituents_info.groupby(['Sector'])['Sector'].count().sort_values(ascending=False).plot.bar()

In [71]:
def column_to_list_from_df(dataframe, column_name):

    """
    Retrieve the values from a specific column in a DataFrame and convert them to a list.

    Args:
        dataframe (pandas.DataFrame): The DataFrame containing the desired column.
        column_name (str): The name of the column to extract values from.

    Returns:
        list: A list containing the values from the specified column.

    Raises:
        KeyError: If the specified column does not exist in the DataFrame.
        Exception: If an error occurs while retrieving the column values.
        
    """

    try:
        output = dataframe[column_name].values.tolist()
        return output
    except KeyError:
        raise KeyError(f"Column '{column_name}' does not exist in the DataFrame.")
    except:
        raise Exception("An error occurred while retrieving the column values.")


In [72]:
#A list of tickers from contituents
dax_tickers = column_to_list_from_df(dataframe=dax_constituents_info, column_name='Ticker')

**2.Download monthly data from yahoo API**

In [74]:
def get_single_stock_close_prices(ticker, start_date="2001-04-01", end_date="2023-01-01", price_interval='1mo'):

    """
    Retrieve historical stock close prices for a given ticker symbol within a specified time period and price interval.

    Args:
        ticker (str): Ticker symbol of the stock.
        start_date (str, optional): Start date of the desired time period in 'YYYY-MM-DD' format. Default is '2001-04-01'.
        end_date (str, optional): End date of the desired time period in 'YYYY-MM-DD' format. Default is '2023-01-01'.
        price_interval (str, optional): Interval at which to retrieve stock prices. Acceptable values are: '1m', '2m', '5m',
            '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'. Default is '1mo'.

    Returns:
        pandas.Series: Series containing the closing stock prices.

    Raises:
        ValueError: If the specified price interval is not in the list of acceptable interval types.
        ValueError: If the stock either does not exist or does not have data for the given time period.
    
    """


    interval_types = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']

    if price_interval not in interval_types:
        raise ValueError(f'Unknown interval type: {price_interval}. Acceptable arguments are {interval_types}')
    

    stock_data = yf.Ticker(ticker)
    stock_series = stock_data.history(start=start_date, end=end_date, interval=price_interval).Close

    if stock_series.empty:
        raise ValueError(f'{ticker} either does not exist or does not have data for a given time period')

    return stock_series


In [75]:
def convert_datetime_index_to_date(dataframe):

    """
    Convert the datetime index of a DataFrame to date only.

    Args:
        dataframe (pandas.DataFrame): The DataFrame to convert.

    Returns:
        pandas.DataFrame: The DataFrame with the datetime index converted to date only.

    Raises:
        Exception: If an AttributeError occurs during the conversion process.
    
    """    

    try:
        dataframe.index = pd.to_datetime(dataframe.index.date)
    except AttributeError as e:
        raise Exception(f'Could not convert index of a dataframe to date. Error message: {e}')
    return dataframe

In [76]:
def get_batch_stock_close_prices(tickers):

    """
    Retrieve historical stock price data for a list of tickers.

    Args:
        tickers (list): List of ticker symbols for the stocks.

    Returns:
        pandas.DataFrame: DataFrame containing historical stock price data for the given tickers.

    """

    output = {}
    tickers_downloaded = []

    with tqdm(total = len(tickers)) as pbar:  # controls the process bar (context manager)
        for single_ticker in tickers:
            pbar.set_postfix_str(single_ticker)
            try:
                stock_price = get_single_stock_close_prices(ticker=single_ticker)
                output[single_ticker] = stock_price  # save pandas.series in a dictionary
                tickers_downloaded.append(single_ticker)  # save successfully downloaded tickers
            except ValueError as e:
                logging.warning(f'Non-existent ticker {single_ticker} or no data for given time interval')
                pass
            pbar.update()
            time.sleep(0.5)
    
    output = pd.DataFrame(output)
    output = convert_datetime_index_to_date(output)

    logging.info(f'Successfully downloaded stocks {tickers_downloaded}')
    
    return output

In [77]:
dax_monthly_prices = get_batch_stock_close_prices(tickers=dax_tickers)

100%|██████████| 40/40 [00:32<00:00,  1.22it/s, ZAL.DE] 


In [80]:
def drop_columns_with_na(dataframe):

    """
    Drop columns from a DataFrame that contain any missing values (NA).

    Args:
        dataframe (pandas.DataFrame): The DataFrame to process.

    Returns:
        pandas.DataFrame: The DataFrame with columns containing missing values dropped.

    Raises:
        Exception: If an error occurs while dropping columns with NA.

    """

    try:
        columns_contain_na = dataframe.columns[dataframe.isnull().any()].tolist()
        df_dropped = dataframe.drop(columns=columns_contain_na)
    except:
        raise Exception('Could not drop columns with NA')

    logging.info(f'{(columns_contain_na)} contain NA values. They are dropped from the dataframe')

    return df_dropped

In [81]:
dax_monthly_prices_clean = drop_columns_with_na(dataframe=dax_monthly_prices)

In [82]:
def price_to_returns(dataframe_prices):
    
    """
    Calculate the returns for stock prices.

    Args:
        df (pandas.DataFrame): A DataFrame containing stock prices.
            The DataFrame should have numerical columns representing the stock prices.

    Returns:
        pandas.DataFrame: A new DataFrame containing the calculated returns.
            The returned DataFrame will have the same columns as the input DataFrame,
            but with the first row dropped since it contains NaN values.
    """
    
    df_returns = dataframe_prices.pct_change().dropna()

    return df_returns


In [83]:
dax_monthly_returns = price_to_returns(dataframe_prices=dax_monthly_prices_clean)

In [85]:
# print(dax_monthly_returns)

**3. Miniumum variance portfolio** <br>
*Machine learning approach - tuning hyperparameters to achieve portfolio with least variance*

In [96]:
def mvp_name(cov_est, mu_est, penalty):
    """
    Generates a name for a minimum variance portfolio based on the given covariance estimation method, mean estimation method, and penalty value.

    Args:
        cov_est (str): The covariance estimation method.
        mu_est (str): The mean estimation method.
        penalty (float): The penalty value.

    Returns:
        str: The generated portfolio name.

    Raises:
        KeyError: If the provided covariance or mean estimation method is not implemented.
    """
    cov_est_encoding = {
        "sample_cov": 'sample',
        "semicovariance": 'semi',
        "exp_cov": 'exp',
        "ledoit_wolf": 'lw',
        "ledoit_wolf_constant_variance": 'lwcv',
        "ledoit_wolf_single_factor": 'lwsf',
        "ledoit_wolf_constant_correlation": 'lwcc',
        "oracle_approximating": 'oa'
    }

    mu_est_encoding = {
        "mean_historical_return": 'mean',
        "ema_historical_return": 'ema',
        "capm_return": 'capm'
    }

    mvp_name_template = 'mvp_{cov_est_name}_{mu_est_name}_{penalty_value}'

    try:
        mvp_name_filled = mvp_name_template.format(
            cov_est_name=cov_est_encoding[cov_est],
            mu_est_name=mu_est_encoding[mu_est],
            penalty_value=penalty
        )
        return mvp_name_filled

    except KeyError as k:
        raise KeyError(f'{k} is not implemented')


In [97]:
def mvp_weights(dataframe_returns, cov_est="sample_cov", mu_est="mean_historical_return", penalty=0):
    """
    Computes the minimum variance portfolio weights.

    Parameters:
        dataframe_returns (pandas.DataFrame): DataFrame containing asset returns.
        cov_est (str, optional): Method to estimate the covariance matrix. Defaults to "sample_cov".
        mu_est (str, optional): Method to estimate expected returns. Defaults to "mean_historical_return".
        penalty (float, optional): L2 regularization penalty. Defaults to 0.

    Returns:
        dict or None: Dictionary of asset weights for the minimum variance portfolio or None if an error occurs.

    Raises:
        ValueError: If the DataFrame has less than 2 columns.
        Exception: If an error occurs during the computation, an exception is raised with an error message.
    """
    if dataframe_returns.shape[1] < 2:
        raise ValueError("DataFrame must have at least 2 columns.")

    try:
        mu = expected_returns.return_model(prices=dataframe_returns, returns_data=True, frequency=12, method=mu_est)  # estimates for expected returns
        cov_mat = risk_models.risk_matrix(prices=dataframe_returns, returns_data=True, frequency=12, method=cov_est)  # estimates for covariance matrix
        cov_mat_fix = risk_models.fix_nonpositive_semidefinite(matrix=cov_mat)  # fix matrix if it's non-positive semidefinite

        ef = EfficientFrontier(expected_returns=mu, cov_matrix=cov_mat_fix)
        ef.add_objective(objective_functions.L2_reg, gamma=penalty)
        ef.min_volatility()
        weights = ef.clean_weights()
        return weights
    
    except Exception as e:
        raise Exception(f'Error message: {e}')

In [98]:
def calculate_returns(portfolio_weights, stock_returns):
    """
    Calculate the portfolio return.

    Args:
        portfolio_weights (list or array-like): The weights of the assets in the portfolio.
        stock_returns (pandas.Series): The returns of the stocks in the portfolio.

    Returns:
        float or None: The calculated portfolio return. If an error occurs during calculation,
                      None is returned.

    """
    try:
        np_weights = pd.Series(portfolio_weights).sort_index().to_numpy()
        np_returns = stock_returns.sort_index().to_numpy()
        portfolio_return = np.dot(np_weights, np_returns)
        return portfolio_return
    except (AttributeError, KeyError, TypeError) as e:
        print(f"An error occurred while calculating portfolio returns: {e}")
        return None

In [127]:
def mvp_oos_returns(dataframe_returns, is_period=24, cov_est="sample_cov", mu_est="mean_historical_return", penalty=0):
    """
    Calculates the out-of-sample portfolio returns based on the given historical returns data and parameters.

    Args:
        dataframe_returns (pandas.DataFrame): Historical returns data as a DataFrame.
        is_period (int, optional): Length of the in-sample period. Defaults to 24.
        cov_est (str, optional): Covariance estimation method. Defaults to "sample_cov".
        mu_est (str, optional): Mean estimation method. Defaults to "mean_historical_return".
        penalty (float, optional): Penalty value. Defaults to 0.

    Returns:
        pandas.Series: Out-of-sample portfolio returns.

    Raises:
        KeyError: If the provided covariance or mean estimation method is not implemented.
    """
    port_name = mvp_name(cov_est=cov_est, mu_est=mu_est, penalty=penalty)

    n = dataframe_returns.shape[0]  # number of observations - 260
    n_is = is_period  # length of in-sample period - 24
    n_oos = n - n_is # length of out-of-sample period - 236

    date_oos = dataframe_returns.iloc[n_is:n].index
    port_oos_returns = []

    for i in range(n_oos):
            
        index_is = list(range(i, i + n_is))
        index_oos = n_is + i

        stock_returns_is = dataframe_returns.iloc[index_is,]
        stock_returns_oos = dataframe_returns.iloc[index_oos]

        #if error in calculating weights (due to problems with covariance matrix), assign zero as returns
        try:
            port_weights = mvp_weights(dataframe_returns=stock_returns_is, cov_est=cov_est, mu_est=mu_est, penalty=penalty)
            port_return = calculate_returns(portfolio_weights=port_weights, stock_returns=stock_returns_oos)
            port_oos_returns.append(port_return)
        except:
            port_return = 0
            port_oos_returns.append(port_return)
            
    port_oos_returns = pd.Series(data=port_oos_returns, index=date_oos, name=port_name)

    return port_oos_returns


In [121]:
def mvp_tune(dataframe_returns, is_period=24, parameter_values = {'cov_est':["sample_cov"], 'mu_est':['mean_historical_return'], 'penalty':[0]}):
    """
    Tune minimum variance portfolio based on different parameter combinations.

    Parameters:
        dataframe_returns (DataFrame): Historical returns data.
        is_period (int): Number of periods.
        param_values (dict): Dictionary of parameter values.

    Returns:
        DataFrame: Results of the tuned portfolios.

    """
    estimation_progress_template = 'Covariance matrix: {cov_est} | Expected retuns: {mu_est} | Penalty value: {penalty}'  # holds info on calculated portfolio
    parameter_combinations = list(itertools.product(*parameter_values.values()))
    output = {}

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")

        with tqdm(total = len(parameter_combinations)) as pbar:
            
            for parameters in parameter_combinations:
                parameters_selected = dict(zip(parameter_values.keys(), parameters))
                estimation_progress = estimation_progress_template.format(cov_est = parameters_selected['cov_est'],
                                                    mu_est = parameters_selected['mu_est'],
                                                    penalty = parameters_selected['penalty'])
                
                pbar.set_postfix_str(estimation_progress)
                
                try:
                    port_returns = mvp_oos_returns(dataframe_returns=dataframe_returns, is_period=is_period, **parameters_selected)
                    output[port_returns.name] = port_returns
                except Exception as e:
                    print(f"Error occurred for hyperparameters: {parameters_selected}. Error: {str(e)}")
                
                pbar.update()

    df_results = pd.DataFrame(output)

    return df_results

In [128]:
hyperparameter_values = {
    'cov_est': ["sample_cov", "semicovariance", "ledoit_wolf", "oracle_approximating"],
    'mu_est': ["mean_historical_return"],
    'penalty': [0, 0.25, 0.5, 1, 2, 4, 8]  # [0, 0.25, 0.5, 1, 2, 4, 8]
}

In [123]:
mvp_candidate_returns = mvp_tune(dataframe_returns=dax_monthly_returns, is_period = 24, parameter_values=hyperparameter_values)

100%|██████████| 28/28 [09:34<00:00, 20.53s/it, Covariance matrix: oracle_approximating | Expected retuns: mean_historical_return | Penalty value: 8]   


In [ ]:
display(mvp_candidate_returns)

**4. Plot results**

In [206]:
def annual_expected_returns(monthly_returns):
    """
    Calculate the annual expected returns from monthly returns.

    Parameters:
        monthly_returns (array-like): A sequence of monthly returns.

    Returns:
        float or None: The annual returns calculated from the mean of monthly returns, rounded to four decimal places.
                       Returns None if an error occurs during the calculation.

    Raises:
        None.
    """
    try:
        return_pa = monthly_returns.mean() * 12
        return_pa_round = np.round(return_pa, 4)
        return return_pa_round
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [207]:
def annual_volatility(monthly_returns):

    """
    Calculate the annual volatility from monthly returns returns.

    Parameters:
        monthly_returns (array-like): A sequence of monthly stock returns.

    Returns:
        float or None: The annual volatility of the stock returns, rounded to four decimal places.
                       Returns None if an error occurs during the calculation.

    Raises:
        ValueError: If the length of monthly_returns is less than 2.
    """

    if len(monthly_returns) < 2:
            raise ValueError("At least two stock returns are required to calculate volatility.")    
    try:
        volatility_pa = monthly_returns.std()*np.sqrt(12)
        volatility_pa_round = np.round(volatility_pa, 4)
        return volatility_pa_round
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [208]:
def risk_metrics(monthly_returns):
    """
    Calculate risk metrics for a given set of monthly returns.

    Args:
        monthly_returns (pandas.DataFrame): DataFrame containing the monthly returns for different portfolios.

    Returns:
        pandas.DataFrame: DataFrame containing the calculated risk metrics, including expected returns and volatility.
                          The DataFrame has the following structure:
                          - Column 1: 'portfolio_name'
                          - Column 2: 'expected_returns'
                          - Column 3: 'volatility'
    """
    expected_returns_pa = {col: annual_expected_returns(monthly_returns[col]) for col in monthly_returns.columns}
    volatility_pa = {col: annual_volatility(monthly_returns[col]) for col in monthly_returns.columns}

    metrics = {"expected_returns": expected_returns_pa, "volatility": volatility_pa}
    metrics_df = pd.DataFrame(metrics)
    
    metrics_df.index.name = "name"
    metrics_df = metrics_df.reset_index()

    return metrics_df

In [209]:
mvp_candidate_metrics = risk_metrics(monthly_returns=mvp_candidate_returns)

In [251]:
mvp_best = mvp_candidate_metrics[mvp_candidate_metrics.volatility == mvp_candidate_metrics.volatility.min()]

In [ ]:
# #portfolio with least volatility
# mvp_best = df_mvp[df_mvp.volatility == df_mvp.volatility.min()]

In [210]:
dax_metrics = risk_metrics(monthly_returns=dax_monthly_returns.loc[mvp_candidate_returns.index,])

In [212]:
dax_constituents_metrics = dax_metrics.merge(dax_constituents_info, how = "left", left_on = "name", right_on = "Ticker")

In [214]:
print(dax_constituents_metrics.head(5))

      name  expected_returns  volatility   Ticker     Name              Sector
0   ADS.DE            0.1466      0.2640   ADS.DE   Adidas               Other
1   AIR.DE            0.2156      0.3476   AIR.DE   Airbus               Other
2   ALV.DE            0.1399      0.2787   ALV.DE  Allianz  Financial Services
3   BAS.DE            0.1224      0.2595   BAS.DE     BASF     Basic Materials
4  BAYN.DE            0.1208      0.2562  BAYN.DE    Bayer          Healthcare


In [301]:
fig = px.scatter(
    dax_constituents_metrics, 
    x = "volatility", 
    y = "expected_returns", 
    color = "Sector",
    color_discrete_sequence=px.colors.qualitative.Set1,
    labels={"expected_returns": "Expected returns p.a.", "volatility": "Volatility p.a."},
    hover_name = "Name",
    hover_data={"expected_returns":True, "volatility":True,"Sector":False})
fig.update_layout(
    title_text="Out-of-sample Risk-return matrix", 
    title_x=0.5, 
    font={'size': 15}, 
    hoverlabel = {"font_size": 15}, 
    width=1200, 
    height=700)
fig.add_trace(
    go.Scatter(
        x=mvp_best.volatility, 
        y=mvp_best.expected_returns,
        mode='markers',
        marker_symbol="star",
        name='Minimum Variance Portfolio',
        marker=dict(line=dict(color="black", width=3)),
        hovertemplate='<b>Minimum variance portfolio</b> <br><br>Volatility p.a.=%{x} <br>Expected returns p.a.=%{y}'
        ))
fig.update_traces(marker_size=30)
fig.show()